[Reference](https://pub.towardsai.net/step-by-step-approach-to-building-a-recommendation-system-a65be5a54045)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Medium

Mounted at /content/drive
/content/drive/MyDrive/Medium


In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

books = pd.read_csv('Books.csv')
ratings = pd.read_csv('https://raw.githubusercontent.com/gowthamsr37/Book-Recommendation-system/main/Ratings.csv')
users = pd.read_csv('https://raw.githubusercontent.com/gowthamsr37/Book-Recommendation-system/main/Users.csv')

In [3]:
books = books[['ISBN', 'Book-Title']]

In [4]:
print(books.shape) , print(users.shape) ,print(ratings.shape)

(271360, 2)
(278858, 3)
(1149780, 3)


(None, None, None)

In [5]:
book_ratings = books.merge(ratings ,on = 'ISBN')
user_rating = users.merge(ratings , on = 'User-ID')

In [6]:
book_num_ratings = book_ratings.groupby('Book-Title')['Book-Rating'].count().reset_index().rename(columns = {'Book-Rating':'Num-Ratings' })
book_avg_ratings = book_ratings.groupby('Book-Title')['Book-Rating'].mean().reset_index().rename(columns = {'Book-Rating':'Avg-Ratings' })
final_rating = book_num_ratings.merge(book_avg_ratings , on = 'Book-Title')

# Collaborative Filtering

In [7]:
popular_books = final_rating[final_rating['Num-Ratings'] > 250].sort_values(by = 'Avg-Ratings'  , ascending= False).reset_index(drop = True).head(50)

popular_books.head()

,Book-Title,Num-Ratings,Avg-Ratings
0,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804
1,Harry Potter and the Goblet of Fire (Book 4),387,5.824289
2,Harry Potter and the Sorcerer's Stone (Book 1),278,5.737410
3,Harry Potter and the Order of the Phoenix (Boo...,347,5.501441
4,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453


In [8]:
x = book_ratings.groupby('User-ID').count()['Book-Rating'] > 200
educated_users  = x[x].index

book_ratings = book_ratings[book_ratings['User-ID'].isin(educated_users)]

y  = book_ratings.groupby('Book-Title')['Book-Rating'].count() >= 50
famous_books = y[y].index

final = book_ratings[book_ratings['Book-Title'].isin(famous_books)]

In [9]:
pt = final.pivot_table(index = 'Book-Title' , columns = 'User-ID' , values= 'Book-Rating').fillna(0)

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
similarity_scores = cosine_similarity(pt)

In [12]:
sorted(list(enumerate(similarity_scores[0])) , key=lambda x: x[1] , reverse= True)[1:6]

[(47, 0.2702651417103732),
 (545, 0.2639619371123496),
 (82, 0.2366937434740099),
 (634, 0.23299389358170397),
 (551, 0.2262639743141286)]

In [13]:
def recommend(book_name):
    index = np.where(pt.index == book_name)[0][0]
    similar_books = sorted(enumerate(similarity_scores[index]),key= lambda x: x[1], reverse =True)[1:6]
    
    for i in similar_books:
        print(pt.index[i[0]])

In [14]:
recommend("4 Blondes")

The House of the Spirits
Pride and Prejudice
Pleading Guilty
Seabiscuit
Notes from a Small Island


In [15]:
recommend("The House of the Spirits")

You Belong To Me
Last Chance Saloon
The Lost World
The Cradle Will Fall
I'll Be Seeing You


In [17]:
# from flask import Flask, render_template, request
# import pickle
# import pandas as pd
# import numpy as np

# app = Flask(__name__)

# popular_df = pickle.load(open('popular_books.pkl', 'rb'))
# pt = pickle.load(open('pt.pkl', 'rb'))
# books = pickle.load(open('books.pkl', 'rb'))
# similarity_scores = pickle.load(open('similarity_scores.pkl', 'rb'))
# book_titles = list(pt.index.values)

# @app.route('/')
# def index():
#     return render_template('index.html',
#                            book_name=list(popular_df['Book-Title'].values),
#                            author=list(popular_df['Book-Author'].values),
#                            image=list(popular_df['Image-URL-M'].values),
#                            votes=list(popular_df['num_of_rating'].values),
#                            rating=list((round(popular_df['avg_rating'], 2).values)))


# @app.route('/recommend')
# def recommend_ui():
#     return render_template('recommend.html')

# @app.route('/recommend_books',methods=['GET','POST'])
# def recommend():
#     user_input = request.form.get('user_input')
#     index = np.where(pt.index == user_input)[0][0]
#     similar_items = sorted(list(enumerate(similarity_scores[index])), key=lambda x: x[1], reverse=True)[1:5]

#     data = []
#     for i in similar_items:
#         item = []
#         temp_df = books[books['Book-Title'] == pt.index[i[0]]]
#         item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
#         item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
#         item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))

#         data.append(item)

#     print(data)

#     return render_template('recommend.html',book_titles=book_titles, data=data)

# if __name__ == '__main__':
#     app.run(debug=True)